In [1]:
import pandas as pd
import os
import ast
import numpy as np
import pickle
from datetime import datetime

In [2]:
path = 'C:/Users/HEN1/Projects/YELP_DATASET/DATA'
file_names = ['yelp_academic_dataset_business.json', 
              'yelp_academic_dataset_checkin.json',
              'yelp_academic_dataset_review.json', 
              'yelp_academic_dataset_tip.json', 
              'yelp_academic_dataset_user.json', 
              'yelp_academic_dataset_review_notext.csv']
files = [os.path.join(path, f) for f in file_names]

In [3]:

df_business = pd.read_json(files[0], lines=True)
#df_checkin = pd.read_json(files[1], lines=True)
#df_tip = pd.read_json(files[3], lines=True)


In [4]:

# An error in dataset
df_business.loc[129241,'state'] = 'BC'

In [5]:
df_review = pd.read_json(files[2], lines=True)
df_review = df_review.drop(columns=['useful', 'funny', 'cool', 'text'])
output_file = 'yelp_academic_dataset_review_notext.csv'
output = os.path.join(path, output_file)
df_review.to_csv(output)

KeyboardInterrupt: 

In [5]:
df_review = pd.read_csv(files[5], index_col=0, low_memory=True)

C:\Users\HEN1\miniconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
df_user = pd.read_json(files[4], lines=True)

In [7]:
# Get restaurants in business dataset
df_rest = df_business[df_business['categories'].str.contains('Restaurant.*')==True].reset_index()
df_rest = df_rest.drop(columns = 'index')

In [8]:
df_rest_name = df_rest[['business_id', 'name']]
df_rest_name

,business_id,name
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX
2,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food
3,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit
4,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs
...,...,...
50788,Zl6SUy6x9jqjRu2HbtEO6A,Split Dine and Drink
50789,Q78fYV6B6P6GmX07YVgi4g,Maudie’s Hill Country
50790,uXdQkuEtvLAzfc3MsO-sTQ,Mama's Cocina Latina
50791,GB75wPibj3IjNauaoCxyGA,Havana Cafe


In [9]:
df_rest = pd.concat([df_rest.drop(['attributes'], axis=1), df_rest['attributes'].apply(pd.Series)], axis=1)

In [10]:
df_rest = df_rest.dropna(axis=1,thresh=len(df_rest)*0.1)

In [11]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50793 entries, 0 to 50792
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 50793 non-null  object 
 1   name                        50793 non-null  object 
 2   address                     50793 non-null  object 
 3   city                        50793 non-null  object 
 4   state                       50793 non-null  object 
 5   postal_code                 50793 non-null  object 
 6   latitude                    50793 non-null  float64
 7   longitude                   50793 non-null  float64
 8   stars                       50793 non-null  float64
 9   review_count                50793 non-null  int64  
 10  is_open                     50793 non-null  int64  
 11  categories                  50793 non-null  object 
 12  hours                       42904 non-null  object 
 13  RestaurantsTableService     191

In [12]:
df_rest['BusinessParking']=df_rest['BusinessParking'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)

In [13]:
df_rest = pd.concat([df_rest.drop(['BusinessParking'], axis=1), df_rest['BusinessParking'].apply(pd.Series)], axis=1)

In [14]:
df_rest = df_rest.drop(columns = 0)

In [15]:
df_rest = pd.concat([df_rest.drop(['hours'], axis=1), df_rest['hours'].apply(pd.Series)], axis=1)

In [16]:
df_rest['Ambience']=df_rest['Ambience'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)
df_rest['GoodForMeal']=df_rest['GoodForMeal'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)

In [17]:
df_rest = pd.concat([df_rest.drop(['Ambience'], axis=1), df_rest['Ambience'].apply(pd.Series)], axis=1)
df_rest = pd.concat([df_rest.drop(['GoodForMeal'], axis=1), df_rest['GoodForMeal'].apply(pd.Series)], axis=1)

In [18]:
df_rest['Music']=df_rest['Music'].apply(lambda x: ast.literal_eval(x) if (np.all(pd.notnull(x))) else x)
df_rest = pd.concat([df_rest.drop(['Music'], axis=1), df_rest['Music'].apply(pd.Series)], axis=1)

In [19]:
df_rest = df_rest.drop(columns = 0)

In [20]:
df_categories_dummies = pd.Series(df_rest['categories']).str.get_dummies(',')
df_categories_dummies = df_categories_dummies.astype('category')
df_rest = pd.concat([df_rest.drop(['categories'], axis=1), df_categories_dummies], axis=1)

In [21]:
tf_col = ['RestaurantsTableService', 'BikeParking', 'BusinessAcceptsCreditCards', 'RestaurantsReservations', 'WheelchairAccessible', 'Caters',
          'OutdoorSeating', 'RestaurantsGoodForGroups', 'HappyHour', 'BusinessAcceptsBitcoin', 'HasTV', 'DogsAllowed', 
          'RestaurantsTakeOut','RestaurantsDelivery', 'GoodForKids', 'lot', 'street', 'valet', 'validated',
          'casual', 'classy', 'divey', 'hipster', 'intimate', 'romantic', 'touristy', 'trendy', 'upscale', 'breakfast', 
          'brunch', 'dessert', 'dinner', 'latenight', 'lunch', 'background_music', 'dj', 'jukebox', 'karaoke', 'live', 'no_music', 
          'video', 'garage'] 
cat_col = list(df_rest.select_dtypes(include=['object']).columns)

In [22]:
d = {'True': 'True', 'False': 'False', 'None':np.nan, True: 'True', False: 'False'}
for col in tf_col: 
    df_rest[col] = df_rest[col].map(d)


In [23]:
week_col = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_rest = df_rest.drop(columns=week_col)

In [24]:
wifi_d = {"u'free'": 'free', "'free'": 'free', "u'paid'": 'paid', "'paid'": 'paid', "u'no'": 'no', "'no'": 'no', 'None': np.nan}
df_rest['WiFi'] = df_rest['WiFi'].map(wifi_d)

In [25]:
Alcohol_d = {"'beer_and_wine'": 'beer_and_wine', "u'beer_and_wine'": 'beer_and_wine', 
             "'full_bar'": 'full_bar', "u'full_bar'": 'full_bar', 
             "u'none'": 'no', "'none'": 'no', 
             'None': np.nan}
df_rest['Alcohol'] = df_rest['Alcohol'].map(Alcohol_d)

In [26]:
NoiseLevel_d = {"u'average'": 'average', "'average'": 'average', 
                "u'quiet'": 'quiet', "'quiet'": 'quiet', 
                "'loud'": 'loud', "u'loud'": 'loud', 
                "u'very_loud'":'very_loud' , "'very_loud'": 'very_loud',
                'None': np.nan}
df_rest['NoiseLevel'] = df_rest['NoiseLevel'].map(NoiseLevel_d)

In [27]:
RestaurantsAttire_d = {"'casual'": 'casual', "u'casual'": 'casual', 
                       "u'dressy'": 'dressy', "'dressy'": 'dressy', 
                       "u'formal'": 'formal', "'formal'": 'formal', 
                       'None': np.nan}
df_rest['RestaurantsAttire'] = df_rest['RestaurantsAttire'].map(RestaurantsAttire_d)

In [28]:
RestaurantsPriceRange2_d = {1.0:1, 2.0:2, 3.0:3, 4.0:4, '1':1, '2':2, '3':3, '4':4, 'None':np.nan}
df_rest['RestaurantsPriceRange2'] = df_rest['RestaurantsPriceRange2'].map(RestaurantsPriceRange2_d)

In [29]:
df_rest['is_open'] = df_rest['is_open'].astype('category')

In [30]:
df_rest = df_rest.drop(columns=['name', 'address'])

In [31]:
df_rest_van_id = df_rest[df_rest['state']=='BC'][['business_id']]

In [32]:
df_rest['new_business_id'] = np.arange(len(df_rest))
df_rest_id = df_rest[['business_id', 'new_business_id']]
df_rest_id

C:\Users\HEN1\AppData\Local\Temp/ipykernel_20036/1529056091.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_rest['new_business_id'] = np.arange(len(df_rest))


,business_id,new_business_id
0,6iYb2HFDywm3zjuRg0shjw,0
1,tCbdrRPZA0oiIYSmHG3J0w,1
2,D4JtQNTI4X3KcbzacDJsMw,2
3,jFYIsSb7r1QeESVUnXPHBw,3
4,HPA_qyMEddpAEtFof02ixg,4
...,...,...
50788,Zl6SUy6x9jqjRu2HbtEO6A,50788
50789,Q78fYV6B6P6GmX07YVgi4g,50789
50790,uXdQkuEtvLAzfc3MsO-sTQ,50790
50791,GB75wPibj3IjNauaoCxyGA,50791


In [33]:
df_rest_van_id = df_rest[df_rest['state']=='BC'][['business_id']]

In [34]:
df_rest = df_rest.rename(columns={'stars':'business_stars'})

In [35]:
df_review_c = df_review.copy()

In [36]:
df_review_c = df_review_c[df_review_c['business_id'].isin(df_rest_van_id['business_id'])]

In [37]:
df_review_c = df_review_c.join(df_rest.set_index('business_id'), on='business_id')

In [38]:
df_review_c['date'] = pd.to_datetime(df_review_c['date'], format='%Y%m%d %H:%M:%S')

In [39]:
df_review_c = df_review_c.drop(columns = 'business_id')

In [40]:
df_user = df_user[['user_id', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'fans', 'average_stars']]
df_user['elite'] = df_user.elite.str.split(',')
df_user['yelping_since'] = pd.to_datetime(df_user['yelping_since'], format='%Y%m%d %H:%M:%S')
df_user = df_user.rename(columns={'review_count':'user_review_count'})
df_user['new_user_id'] = np.arange(len(df_user))

In [41]:
df_review_c = df_review_c.join(df_user.set_index('user_id'), on='user_id')
is_elite = df_review_c.apply(lambda row: 1 if (str(row['date'].year) in row['elite']) else 0, axis =1)
is_elite = is_elite.astype('category')
df_review_c = pd.concat([df_review_c.drop(['elite'], axis=1), is_elite], axis=1)
df_review_c = df_review_c.rename(columns={0:'is_elite'})

In [42]:
df_review_c = df_review_c.drop(columns = 'user_id')

In [43]:
df_review_c = df_review_c.drop(columns=['review_id', 'state'])

In [44]:
obj_col = df_review_c.select_dtypes(include=['object']).columns
cat = df_review_c[obj_col].astype('category')
df_review_c[obj_col] = cat

In [45]:
df_review_c = df_review_c.loc[:,df_review_c.apply(pd.Series.nunique) > 1]

In [46]:
city_d = {"BURNABY": 'Burnaby', "N Vancouver": 'North Vancouver', "NEW WESTMINSTER": 'New Westminster',
          "New Westminister": 'New Westminster', "Newwestminster": 'New Westminster', "Port coquitlam": 'Port Coquitlam', 
          'RICHMOND': 'Richmond', 'RichMond': 'Richmond','SURREY': 'Surrey'}
for key in list(city_d.keys()):
    df_review_c.loc[df_review_c.city==key, 'city']=city_d[key]

In [47]:
df_review_c

,stars,date,city,postal_code,latitude,longitude,business_stars,review_count,is_open,RestaurantsTableService,...,Wineries,Women's Clothing,new_business_id,user_review_count,yelping_since,useful,funny,cool,new_user_id,is_elite
16,5,2014-02-05 21:09:05,Vancouver,V5L 3X5,49.272800,-123.069730,3.5,225,0,NaN,...,0,0,988,39,2012-08-10 09:41:04,68,38,19,70136,0
17,1,2018-03-03 23:45:25,Vancouver,V6E 1A1,49.284877,-123.122629,4.0,849,1,NaN,...,0,0,4629,70,2009-12-11 18:25:05,118,26,21,18046,0
22,5,2016-01-30 02:39:22,Vancouver,V6E 2B1,49.287496,-123.128934,4.5,436,1,True,...,0,0,767,68,2013-06-15 04:55:52,34,10,18,135310,0
40,1,2014-08-10 20:40:15,Vancouver,V6K 1E9,49.270973,-123.155283,3.5,270,1,NaN,...,0,0,1727,2,2012-10-01 04:31:44,2,0,0,67060,0
47,3,2018-09-25 03:22:50,Vancouver,V6B 3A4,49.280443,-123.117036,4.0,2302,1,True,...,0,0,1639,12,2016-05-16 18:49:40,6,1,1,165546,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635186,5,2020-10-07 20:40:26,Vancouver,V5S 4J6,49.215045,-123.030029,4.5,6,1,NaN,...,0,0,46164,100,2016-09-28 04:07:04,177,43,114,302382,0
8635201,5,2020-10-04 19:30:47,Vancouver,V5N 2T7,49.242665,-123.060415,4.0,87,1,True,...,0,0,49268,29,2017-10-01 04:14:27,15,1,7,341238,0
8635342,3,2020-11-22 07:52:59,Burnaby,V5E 0A6,49.218895,-122.956225,2.5,18,1,NaN,...,0,0,50619,12,2018-05-30 00:28:19,0,2,0,641154,0
8635369,4,2021-01-01 19:48:56,North Vancouver,V7M 2E5,49.311679,-123.079106,4.0,87,1,True,...,0,0,45801,62,2016-01-17 01:06:45,30,4,21,143314,0


In [48]:
def trans_timestamp(df:pd.DataFrame, col:str):
    output = pd.DataFrame()
    output[f"{col}_year"] = df[col].dt.year
    output[f"{col}_month"] = df[col].dt.month
    output[f"{col}_day"] = df[col].dt.day
    output[f"{col}_weekday"] = df[col].dt.weekday
    output = pd.concat([df.drop([col], axis=1), output], axis=1)
    return output

In [49]:
df_review_c = trans_timestamp(df_review_c, 'yelping_since')
df_review_c = trans_timestamp(df_review_c, 'date')
df_review_c = df_review_c.drop(['city', 'postal_code'], axis=1)

In [50]:
df_review_c

,stars,latitude,longitude,business_stars,review_count,is_open,RestaurantsTableService,WiFi,BikeParking,BusinessAcceptsCreditCards,...,new_user_id,is_elite,yelping_since_year,yelping_since_month,yelping_since_day,yelping_since_weekday,date_year,date_month,date_day,date_weekday
16,5,49.272800,-123.069730,3.5,225,0,NaN,free,True,NaN,...,70136,0,2012,8,10,4,2014,2,5,2
17,1,49.284877,-123.122629,4.0,849,1,NaN,free,True,NaN,...,18046,0,2009,12,11,4,2018,3,3,5
22,5,49.287496,-123.128934,4.5,436,1,True,free,True,NaN,...,135310,0,2013,6,15,5,2016,1,30,5
40,1,49.270973,-123.155283,3.5,270,1,NaN,no,True,NaN,...,67060,0,2012,10,1,0,2014,8,10,6
47,3,49.280443,-123.117036,4.0,2302,1,True,no,True,True,...,165546,0,2016,5,16,0,2018,9,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635186,5,49.215045,-123.030029,4.5,6,1,NaN,NaN,NaN,NaN,...,302382,0,2016,9,28,2,2020,10,7,2
8635201,5,49.242665,-123.060415,4.0,87,1,True,no,False,NaN,...,341238,0,2017,10,1,6,2020,10,4,6
8635342,3,49.218895,-122.956225,2.5,18,1,NaN,NaN,NaN,NaN,...,641154,0,2018,5,30,2,2020,11,22,6
8635369,4,49.311679,-123.079106,4.0,87,1,True,no,True,NaN,...,143314,0,2016,1,17,6,2021,1,1,4


In [51]:

output_file = 'yelp_str.pkl'
output = os.path.join(path, output_file)
with open(output, 'wb') as file:
    pickle.dump(df_review_c, file)
    

In [52]:
df_rest2 = df_rest.copy()

In [53]:
for col in tf_col: 
    df_dummies = pd.get_dummies(df_rest2[col], prefix=col)
    df_dummies = df_dummies.astype('category')
    df_rest2 = pd.concat([df_rest2.drop([col], axis=1), df_dummies], axis=1)

In [54]:
level_col = ['WiFi', 'Alcohol', 'NoiseLevel', 'RestaurantsAttire', 'RestaurantsPriceRange2' ]

for col in level_col: 
    df_dummies = pd.get_dummies(df_rest2[col], prefix=col)
    df_dummies = df_dummies.astype('category')
    df_rest2 = pd.concat([df_rest2.drop([col], axis=1), df_dummies], axis=1)

In [55]:
df_rest.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50793 entries, 0 to 50792
Columns: 1177 entries, business_id to new_business_id
dtypes: category(1121), float64(4), int32(1), int64(1), object(50)
memory usage: 180.1 MB


In [56]:
df_rest_van_id = df_rest[df_rest['state']=='BC'][['business_id']]

In [57]:
df_review = df_review[df_review['business_id'].isin(df_rest_van_id['business_id'])]

In [58]:
df_review = df_review.join(df_rest2.set_index('business_id'), on='business_id')

In [59]:
df_review['date'] = pd.to_datetime(df_review['date'], format='%Y%m%d %H:%M:%S')

In [60]:
df_review = df_review.drop(columns = 'business_id')

In [61]:
df_user

,user_id,user_review_count,yelping_since,useful,funny,cool,elite,new_user_id
0,q_QQ5kBBwlCcbL1s4NVK3g,1220,2005-03-14 20:26:35,15038,10030,11291,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",0
1,dIIKEfOgo0KqUfGQvGikPg,2136,2007-08-10 19:01:51,21272,10289,18046,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...",1
2,D6ErcUnFALnCQN4b1W_TlA,119,2007-02-07 15:47:53,188,128,130,"[2010, 2011]",2
3,JnPIjvC0cmooNDfsa9BmXg,987,2009-02-09 16:14:29,7234,4722,4035,"[2009, 2010, 2011, 2012, 2013, 2014]",3
4,37Hc8hr3cw0iHLoPzLK6Ow,495,2008-03-03 04:57:05,1577,727,1124,"[2009, 2010, 2011]",4
...,...,...,...,...,...,...,...,...
2189452,OrXXOTSQG2hLEoZ4sw03Gg,1,2018-08-25 23:07:58,0,1,0,[],2189452
2189453,pYZ4Dyx5I92u5gDfGiVTpQ,3,2013-08-17 23:58:29,0,0,0,[],2189453
2189454,DhrXTJRTLhnvI9UI1q63mg,10,2012-10-14 23:35:51,3,0,2,[],2189454
2189455,ka06dBaC9tvKhc7DJ9_7wQ,9,2016-03-24 11:54:52,4,2,0,[],2189455


In [62]:
df_review = df_review.join(df_user.set_index('user_id'), on='user_id')
is_elite = df_review.apply(lambda row: 1 if (str(row['date'].year) in row['elite']) else 0, axis =1)
is_elite = is_elite.astype('category')
df_review = pd.concat([df_review.drop(['elite'], axis=1), is_elite], axis=1)
df_review = df_review.rename(columns={0:'is_elite'})

In [63]:
df_review = df_review.drop(columns = 'user_id')
df_review = df_review.drop(columns=['review_id', 'state'])

In [64]:
obj_col = df_review.select_dtypes(include=['object']).columns
cat = df_review[obj_col].astype('category')
df_review[obj_col] = cat

In [65]:
city_d = {"BURNABY": 'Burnaby', "N Vancouver": 'North Vancouver', "NEW WESTMINSTER": 'New Westminster',
          "New Westminister": 'New Westminster', "Newwestminster": 'New Westminster', "Port coquitlam": 'Port Coquitlam', 
          'RICHMOND': 'Richmond', 'RichMond': 'Richmond','SURREY': 'Surrey'}
for key in list(city_d.keys()):
    df_review.loc[df_review.city==key, 'city']=city_d[key]

In [66]:
df_review = df_review.loc[:,df_review.apply(pd.Series.nunique) > 1]

In [69]:
df_review

,stars,latitude,longitude,business_stars,review_count,is_open,Acai Bowls,Accessories,Active Life,Acupuncture,...,new_user_id,is_elite,yelping_since_year,yelping_since_month,yelping_since_day,yelping_since_weekday,date_year,date_month,date_day,date_weekday
16,5,49.272800,-123.069730,3.5,225,0,0,0,0,0,...,70136,0,2012,8,10,4,2014,2,5,2
17,1,49.284877,-123.122629,4.0,849,1,0,0,0,0,...,18046,0,2009,12,11,4,2018,3,3,5
22,5,49.287496,-123.128934,4.5,436,1,0,0,0,0,...,135310,0,2013,6,15,5,2016,1,30,5
40,1,49.270973,-123.155283,3.5,270,1,0,0,0,0,...,67060,0,2012,10,1,0,2014,8,10,6
47,3,49.280443,-123.117036,4.0,2302,1,0,0,0,0,...,165546,0,2016,5,16,0,2018,9,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635186,5,49.215045,-123.030029,4.5,6,1,0,0,0,0,...,302382,0,2016,9,28,2,2020,10,7,2
8635201,5,49.242665,-123.060415,4.0,87,1,0,0,0,0,...,341238,0,2017,10,1,6,2020,10,4,6
8635342,3,49.218895,-122.956225,2.5,18,1,0,0,0,0,...,641154,0,2018,5,30,2,2020,11,22,6
8635369,4,49.311679,-123.079106,4.0,87,1,0,0,0,0,...,143314,0,2016,1,17,6,2021,1,1,4


In [68]:
df_review = trans_timestamp(df_review, 'yelping_since')
df_review = trans_timestamp(df_review, 'date')
df_review = df_review.drop(['city', 'postal_code'], axis=1)

In [70]:

output_file = 'yelp.pkl'
output = os.path.join(path, output_file)
with open(output, 'wb') as file:
    pickle.dump(df_review, file)
    